## 8Puzzle

<img src='such_bild1.png'>

Wir modellieren eine Spielstellung mit einem Tupel.


In [25]:
start = (7,2,4,5,0,6,8,3,1)
ziel  = (0,1,2,3,4,5,6,7,8)

### BFS - Breitensuche

In [26]:
%%time
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None


def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def nextstates(state):
    tmp = []
    i = state.index(0)
    for d in [-3,3,-1,1]:
        j = i+d
        if 0 <= j < 9 and (i%3==j%3 or i//3==j//3):  # bleibt auf gleicher Zeile oder Spalte
            a = list(state)
            a[i],a[j] = a[j],a[i]
            tmp.append(tuple(a))
    return tmp  
    
def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel


# Aufruf:
prev, goal = bfs(start)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')
print(getMoves(path))

len(prev)=177809
len(path)=27
LURDDLURRULLDRRDLURULDDLUU
CPU times: total: 281 ms
Wall time: 356 ms


### A*



In [27]:
%%time
from heapq import heappop, heappush
def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # die Rückwärtskosten
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            gg = g[state] + 1       
            if v not in prev or gg < g[v]:     
                g[v] = gg                     
                heappush(frontier,(g[v]+h(v),v))  
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def h(state):
    '''
    state: Spielstellung
    returns: Vorwärtskosten laut Heuristik
    '''
    return sum(a!=b for a,b in zip((0,1,2,3,4,5,6,7,8),state))

#Aufruf:
prev, goal = astar(start)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')
print(getMoves(path))
 

len(prev)=46968
len(path)=27
LURDRDLLURRDLLURRULLDRRULL
CPU times: total: 172 ms
Wall time: 200 ms


### Beamsearch

In [31]:
%%time
def beamsearch(startstate, beamwidth): 
    frontier =[startstate]
    prev = {startstate:None}
    nextfrontier = []
    while frontier:
        for state in frontier:
            if goaltest(state):
                return prev, state
            for v in nextstates(state):
                if v not in prev:
                    prev[v] = state
                    nextfrontier.append(v)
        
        frontier = sorted(nextfrontier,key=lambda x: h(x)) 
        frontier = frontier[:beamwidth]
        nextfrontier = []
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

    
#Aufruf:
BEAMWIDTH = 1000                        # bei BEAMWIDTH=900 wird nicht mehr die optimale Lösung gefunden
prev, goal = beamsearch(start,BEAMWIDTH)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')
print(getMoves(path))


len(prev)=23923
len(path)=27
LURDRDLLURRDLLURRULLDRRULL
CPU times: total: 15.6 ms
Wall time: 60.2 ms


In [3]:
def getMove(s1, s2):
    i1 = s1.index(0)   # Position der 0 in state s1  
    i2 = s2.index(0)   # Position der 0 in state s2    
    if i2 == i1-1: return 'L'   #left
    if i2 == i1+1: return 'R'   #right
    if i2 == i1+3: return 'D'   #down
    if i2 == i1-3: return 'U'   #up

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves